In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 12, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'stop', 'logprobs': None}, id='run-a54be865-358d-4613-9678-23502032e2b5-0', usage_metadata={'input_tokens': 12, 'output_tokens': 19, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


In [3]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vz1sES73m9UcQQSxLe3AeJSV', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 135, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0ae931d8-10dd-442e-86c9-d4e535f6e902-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location'

In [5]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-02-18 01:08:09 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vz1sES73m9UcQQSxLe3AeJSV', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 135, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0ae931d8-10dd-442e-86c9-d4e535f6e902-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'locatio

In [6]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 현재 2025년 2월 18일 01시 08분 09초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 192, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6423f78-b1ce-4016-bf64-ff2f2d3646b0-0', usage_metadata={'input_tokens': 192, 'output_tokens': 27, 'total_tokens': 219, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")


In [8]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [9]:
messages.append(HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_3hvAMT8EY9JeBvNWaKJ256A6', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 283, 'total_tokens': 311, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-c531f56b-a2ed-42eb-be54-4600ef8824c7-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_3hvAMT8EY9JeBvNWaKJ256A6', 'type': 'tool_call'}] usage_metadata={'input_tokens': 283, 'output_tokens': 28, 'total_tokens': 311, 'input_token_deta

In [10]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-01-15 00:00:00-05:00 | 409.9  | 429.8  | 405.66 |  428.22 | 8.13755e+07 |           0 |              0 |\n| 2025-01-16 00:00:00-05:00 | 423.49 | 424    | 409.13 |  413.82 | 6.83352e+07 |           0 |              0 |\n| 2025-01-17 00:00:00-05:00 | 421.5  | 439.74 | 419.75 |  426.5  | 9.49914e+07 |           0 |              0 |\n| 2025-01-21 00:00:00-05:00 | 432.64 | 433.2  | 406.31 |  424.07 | 8.73209e+07 |           0 |              0 |\n| 2025-01-22 00:00:00-05:00 | 416.81 | 428    | 414.59 |  415.11 | 6.09633e+07 |           0 |              0 |\n| 2025-01-23 00:00:00-05:00 | 416.06 | 420.73 | 408.95 |  412.38 | 5.06906e+07 |           0 |              0 |\n| 2025-01-24 00:00:00-05:0

In [11]:
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전, 즉 2025년 1월 15일에 테슬라(TSLA)의 종가는 428.22달러였고, 최근인 2025년 2월 14일에는 종가가 355.84달러입니다. \n\n따라서 테슬라의 주가는 한 달 전보다 내렸습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 1642, 'total_tokens': 1720, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'stop', 'logprobs': None}, id='run-c9f5f3e1-4099-4e51-8355-fb5d9642c51a-0', usage_metadata={'input_tokens': 1642, 'output_tokens': 78, 'total_tokens': 1720, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 08-4 스트림 출력

In [12]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점에 대해 이야기해줘.")]):
    print(c.content, end='|') 

|안|녕하세요|!| 한국| 사회|에는| 여러| 가지| 문제|점|이| 있지만|,| 몇| 가지| 주요|한| 이|슈|를| 이야기|해|보|겠습니다|.

|1|.| **|고|용| 문제|**|:| 청|년| 실|업|률|이| 높|고|,| 비|정|규|직| 고|용|이| 증가|하는| 추|세|입니다|.| 많은| 젊|은|이|들이| 안정|적인| 일|자|리를| 찾|기| 어려|워|하고|,| 고|용|의| 질|이| 낮|아|지는| 문제가| 있습니다|.

|2|.| **|소|득| 불|평|등|**|:| 소|득| 분|배|의| 불|균|형|이| 심|화|되고| 있으며|,| 특히| 대|기업|과| 중|소|기업| 간|의| 격|차|가| 크게| 벌|어|지고| 있습니다|.| 이|로| 인해| 사회|적| 계|층| 간|의| 갈|등|이| 심|화|될| 수| 있습니다|.

|3|.| **|주|거|문|제|**|:| 급|격|한| 서울|과| 수도|권|의| 집|값| 상승|으로| 인해| 많은| 사람들이| 주|거| 문제|에| 시|달|리고| 있습니다|.| 특히| 청|년|층|과| 사회|초|년|생|들은| 높은| 집|값|으로| 인해| 주|거| 안정|성을| 확보|하기| 힘|들|어|하고| 있습니다|.

|4|.| **|정|신| 건강| 문제|**|:| 과|도|한| 경쟁|과| 스트|레스|는| 많은| 사람|들에게| 정신|적| 부담|을| 주|며|,| 우|울|증|이나| 불|안|장|애| 같은| 문제가| 증가|하고| 있습니다|.| 이러한| 문제|는| 사회|적으로|도| 큰| 영향을| 미|치|고| 있으|나|,| 여|전히| stig|mat|ization|이| 존재|합니다|.

|5|.| **|성| 평|등| 문제|**|:| 성|별|에| 따른| 임|금| 차|별|이나| 고|용| 불|균|형|,| 가|정| 내| 성|폭|력| 등의| 문제가| 여|전히| 존재|합니다|.| 이러한| 문제|들은| 사회|적| 인|식| 개선|과| 법|적| 제|재|가| 필요|합니다|.

|6|.| **|일|과| 삶|의| 균|형|**|:| 한국|은| 여|전히| 긴| 근|무| 시간|과| 과|도|한| 업무|

In [13]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

response = llm_with_tools.stream(messages)

# 파편화된 tool_call 청크를 하나로 합치기 
is_first = True
for chunk in response:    
    print("chunk type: ", type(chunk))
    
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
    
    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'type': 'tool_c

In [14]:
gathered

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-00266f14-0c23-4644-8306-7c9ae0e11156', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'index': 0, 'type': 'tool_call_chunk'}])

In [15]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 이름으로 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-02-18 01:09:36 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-00266f14-0c23-4644-8306-7c9ae0e11156', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_eScLJfZZaE0kTvF7GgupB0oq', 'index': 0, 'type': 'tool_call_chunk'}]),
 ToolMessage(content='Asia/Seoul (부산) 현재시각 2025-02-18 01

In [16]:
for c in llm_with_tools.stream(messages):
    print(c.content, end='|')

|부|산|은| 지금| |202|5|년| |2|월| |18|일| |01|시| |09|분| |36|초|입니다|.||